In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import pytry

In [5]:
class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),
        
    def evaluate(self, q): #function to fill in missing data points by interpolating
        data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)
        
        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001
        
        def extract_pattern(start, end): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result
        
        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])


        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        t_aim=[]
        p_aim=[]
        
        for i in range(1,(len(start)-1)):
            if label[i]==('noplay'):
                ti, pi = extract_pattern(start[i], end[i])
                t_noplay.append(ti)
                p_noplay.append(pi)
            if label[i]==('goaloriented'):
                ti, pi = extract_pattern(start[i], end[i])
                t_goal.append(ti)
                p_goal.append(pi)
            if label[i]==('aimless'):
                ti, pi = extract_pattern(start[i], end[i])
                t_aim.append(ti)
                p_aim.append(pi)
        
############ randomly split data 80/20 into training and testing sets ############ 
        random.shuffle(p_goal)
        goal_train = p_goal[:(int(len(p_goal)*0.8))] 
        goal_test = p_goal[(int(len(p_goal)*0.8)):]
        
        random.shuffle(p_noplay)
        noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
        noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=2).fit(train_all)

        goal_patterns = {}
        i=0
        for gp in goal_train:
            goal_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_patterns = {}
        i=0
        for npp in noplay_train:
            noplay_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
        noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 
        
############ TRAINING WITH 80% ############ 

        theta = 1.0
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
            rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            
        s_window = int(theta/dt)
        s_pattern = len(min(goal_patterns, key=len))

        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw.inverse_basis(t_window)

        eval_points=[]
        target = []
        for p in range(0,len(goal_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[p], i)[:s_window]))
                target.append([1])
        
        for p in range(0,len(noplay_train)):
            for i in range(s_pattern):
                eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[p], i)[:s_window]))
                target.append([-1])
        eval_points = np.array(eval_points)
        
############ TEST ON TRAINING PATTERNS ############ 

        with net:
            result = nengo.Node(None, size_in=1)
            nengo.Connection(rw.state, result,
                             eval_points=eval_points, scale_eval_points=True,
                             function=target, synapse=0.1)
        
        train_all = np.array(goal_patterns[0])
        for gp in range(1,(len(goal_patterns))):
            train_all = np.hstack([train_all,goal_patterns[gp]])
        for npp in range(0,(len(noplay_patterns))):
            train_all = np.hstack([train_all,noplay_patterns[npp]])

        presentation_time_train=30/len(train_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_train_len = {}
        goal_train_len[0] = int(len(goal_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(goal_train)):
            goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns[i])*presentation_time_train/dt)
        
        goal_train_means = {}
        goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
        for i in range(1,len(goal_train)):
            goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])
            
        noplay_train_len = {}
        noplay_train_len[0] = int(len(noplay_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_train)):
            noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns[i])*presentation_time_train/dt)
        
        noplay_train_means = {}
        noplay_train_means[0] = np.mean(sim.data[p_result][goal_train_len[len(goal_train_len)-1]:noplay_train_len[0]+goal_train_len[len(goal_train_len)-1]])
        for i in range(1,len(noplay_train)):
            noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])
        
        for j in range(len(goal_patterns)):
            key_j = 'gtr_{}'.format(j)  
            goal_train_means[key_j] = goal_train_means.pop(j)
            
        for j in range(len(noplay_patterns)):
            key_j = 'ntr_{}'.format(j)  
            noplay_train_means[key_j] = noplay_train_means.pop(j)

############ EXTRACT 20% TEST PATTERNS ############ 
        goal_test_patterns = {}
        i=0
        for gp in goal_test:
            goal_test_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_test_patterns = {}
        i=0
        for npp in noplay_test:
            noplay_test_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        patterns = {}
        patterns.update(goal_test_patterns)
        patterns.update(noplay_test_patterns)

        goal_test_patterns = [(goal_test_patterns[key]) for key in goal_test_patterns.keys()]
        noplay_test_patterns = [(noplay_test_patterns[key]) for key in noplay_test_patterns.keys()] 
        
        test_all = np.array(goal_test_patterns[0])
        for gp in range(1,(len(goal_test_patterns))):
            test_all = np.hstack([test_all,goal_test_patterns[gp]])
        for npp in range(0,(len(noplay_test_patterns))):
            test_all = np.hstack([test_all,noplay_test_patterns[npp]])
            
        presentation_time_test=30/len(test_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        goal_test_len = {}
        goal_test_len[0] = int(len(goal_test_patterns[0])*presentation_time_test/dt)
        for i in range(1,len(goal_test)):
            goal_test_len[i] = goal_test_len[i-1]+int(len(goal_test_patterns[i])*presentation_time_test/dt)
        
        goal_test_means = {}
        goal_test_means[0] = np.mean(sim.data[p_result][0:goal_test_len[0]])
        for i in range(1,len(goal_test)):
            goal_test_means[i] = np.mean(sim.data[p_result][goal_test_len[i-1]:goal_test_len[i]])
            
        noplay_test_len = {}
        noplay_test_len[0] = int(len(noplay_test_patterns[0])*presentation_time_test/dt)
        for i in range(1,len(noplay_test)):
            noplay_test_len[i] = noplay_test_len[i-1]+int(len(noplay_test_patterns[i])*presentation_time_test/dt)
        
        noplay_test_means = {}
        noplay_test_means[0] = np.mean(sim.data[p_result][goal_test_len[len(goal_test_len)-1]:noplay_test_len[0]+goal_test_len[len(goal_test_len)-1]])
        for i in range(1,len(noplay_test)):
            noplay_test_means[i] = np.mean(sim.data[p_result][noplay_test_len[i-1]:noplay_test_len[i]])
        
        for j in range(len(goal_test)):
            key_j = 'gtst_{}'.format(j)  
            goal_test_means[key_j] = goal_test_means.pop(j)
            
        for j in range(len(noplay_test)):
            key_j = 'ntst_{}'.format(j)  
            noplay_test_means[key_j] = noplay_test_means.pop(j)
        
    
################# TEST ON AIMLESS PATTERNS #################

        random.shuffle(p_aim)
    
        aimless_patterns = {}
        i=0
        for ap in p_aim:
            aimless_patterns[i] = pca_model.transform(ap)[:,0]
            i+=1
        
        test_aim = np.array(aimless_patterns[0])
        for ap in range(1,(len(aimless_patterns))):
            test_aim = np.hstack([test_aim,aimless_patterns[ap]])
        
        presentation_time_test=30/len(test_aim)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ GET RESULTS ############ 

        aim_test_len = {}
        aim_test_len[0] = int(len(aimless_patterns[0])*presentation_time_train/dt)
        for i in range(1,len(p_aim)):
            aim_test_len[i] = aim_test_len[i-1]+int(len(aimless_patterns[i])*presentation_time_train/dt)
        
        aim_test_means = {}
        aim_test_means[0] = np.mean(sim.data[p_result][0:aim_test_len[0]])
        for i in range(1,len(p_aim)):
            aim_test_means[i] = np.mean(sim.data[p_result][aim_test_len[i-1]:aim_test_len[i]])
            
        for j in range(len(p_aim)):
            key_j = 'aim_{}'.format(j)  
            aim_test_means[key_j] = aim_test_means.pop(j)

            
############ SAVE DATA ############ 
        
        means = {**goal_train_means, **noplay_train_means, **goal_test_means, **noplay_test_means, **aim_test_means}
            
        return means


In [6]:
PatternInterpolationTrial().run(data_dir='debug')

running PatternInterpolationTrial#20190228-161657-0b724e1d


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 1
n_neurons = 2000
aim_0 = -0.6410669297401018
aim_1 = -0.771145224967181
aim_10 = -0.8222404224704495
aim_11 = -0.6529332395447003
aim_12 = -0.6788233597390351
aim_13 = -0.6995872033154676
aim_14 = -0.6863016224193571
aim_15 = -0.7163076746496101
aim_16 = -0.7296863481878484
aim_17 = -0.7225334439246696
aim_18 = -0.6631586185620078
aim_19 = -0.6301533985480497
aim_2 = -0.7336100393085713
aim_20 = -0.6473733403825171
aim_3 = -0.7273869255403763
aim_4 = -0.6830039449392407
aim_5 = -0.6938865875024601
aim_6 = -0.7239045176182457
aim_7 = -0.9528168581102453
aim_8 = -0.8990916497713726
aim_9 = -0.6625598832287372
gtr_0 = -0.5295081828126316
gtr_1 = -0.5911858520115587
gtr_2 = -0.6637843867935102
gtr_3 = -0.6987881443258661
gtr_4 = -0.6633578821265366
gtr_5 = -0.6896657007764646
gtr_6 = -0.6903205781030666
gtr_7 = -0.6223191451537249
gtr_8 = -0.6613667942615032
gtr_9 = -0.6657699672575047
gtst_0 = -0.5112231807149745
gtst_1 = -0.6813015252081321
gtst_2 = -0.6892260582940489
ntr_0 = -

{'gtr_0': -0.5295081828126316,
 'gtr_1': -0.5911858520115587,
 'gtr_2': -0.6637843867935102,
 'gtr_3': -0.6987881443258661,
 'gtr_4': -0.6633578821265366,
 'gtr_5': -0.6896657007764646,
 'gtr_6': -0.6903205781030666,
 'gtr_7': -0.6223191451537249,
 'gtr_8': -0.6613667942615032,
 'gtr_9': -0.6657699672575047,
 'ntr_0': -0.646621062798141,
 'ntr_1': -0.6541413136044041,
 'ntr_2': -0.657451990672198,
 'ntr_3': -0.5905690402464812,
 'ntr_4': -0.6522595612973167,
 'ntr_5': -0.582636182768261,
 'ntr_6': -0.4938523715841861,
 'ntr_7': -0.5735626233706648,
 'ntr_8': -0.8441520693043341,
 'ntr_9': -0.6932925090725661,
 'ntr_10': -0.7090433357890579,
 'ntr_11': -0.6298474138277078,
 'ntr_12': -0.5555074616149077,
 'ntr_13': -0.5580806960919878,
 'ntr_14': -0.7732577526506383,
 'ntr_15': -0.730853136727263,
 'ntr_16': -0.6233621651707345,
 'ntr_17': -0.650541600013457,
 'ntr_18': -0.7700067797589654,
 'ntr_19': -0.849571470365395,
 'gtst_0': -0.5112231807149745,
 'gtst_1': -0.6813015252081321,
 '

In [ ]:
for seed in range(20):
    PatternInterpolationTrial().run(seed=seed, data_dir='experiment_childchild')

In [ ]:
%reset